<a href="https://colab.research.google.com/github/PundirShivam/Portfolio/blob/master/Options%26Derivative_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Valuing a call-option using BSM Formulae
# with no dividents
# Here Cases which have closed analytical solutions are considered
# More advanced approaches are discussed elsewhere in respective chapters

In [0]:
def BSM_call_value(So,K,T,r,sigma,q=0):
  '''
  BSM formaule for valuing European call option
  Parameters:
  So : index/stock level today
  K  : strike price of option
  T  : Time to maturity of option (days)
  r  : short term interest rate
  sigma : std. of returns i.e. volatility of index
  q   : q=0 (i.e. divident yield)
  Output:
  Co = call option value today
  '''
  from scipy import stats
  import math
  import numpy as np
  d1 = (np.log(So/K) + (r-q+.5*sigma**2)*T)/(sigma*math.sqrt(T))
  d2 = d1-sigma*math.sqrt(T)
  Co =  So*math.exp(-q*T)*stats.norm.cdf(d1,0,1) - K*math.exp(-r*T)*stats.norm.cdf(d2,0,1)
  return Co
  

In [0]:
# Se^(-q*T) + P = C + Ke^(-rT)
# >> P = C + Ke^(-rT) - Se^(-q*T)
# Using put call parity we can value put option also

def BSM_put_value(So,K,T,r,sigma,q=0):
  '''
  BSM formaule for valuing European put option
  Parameters:
  So : index/stock level today
  K  : strike price of option
  T  : Time to maturity of option (days)
  r  : short term interest rate
  sigma : std. of returns i.e. volatility of index
  q   : q=0 (i.e. divident yield)
  Output:
  Po = call option value today
  '''
  import math
  Co = BSM_call_value(So,K,T,r,sigma,q)
  Po = Co + K*math.exp(-r*T) - So*math.exp(-q*T)
  return Po

In [0]:
print(" Value of Europen Call option is ${0:.2f}".format(BSM_call_value(105,100,1,0.05,.25)))
print(" Value of Europen Put option is ${0:.2f}".format(BSM_put_value(105,100,1,0.05,.25)))

 Value of Europen Call option is $15.65
 Value of Europen Put option is $5.78


In [0]:
## Option Valuation using CRR Cox-Ross-Rubinstein Approach
# i.e. binomial option pricing

def CRR_value_option(So,K,T,r,sigma,M=1000,call=True):
  '''
  Binomial option pricing for valuing European Call Option
  Parameters:
  So : index/stock level today
  K  : strike price of option
  T  : Time to maturity of option (days)
  r  : short term interest rate
  sigma : std. of returns i.e. volatility of index
        Also, constant volatility factor of diffusion
  M : no. of time-steps to be considered till time to maturity
      As M increases the CRR_model value approached BSM value
  call : True will return value of call option
         False will return value of put option
  Output:
  
  Vo = call/put option value today
  '''
  import math
  import numpy as np
  
  # time parameters
  dt = T/ M # length of time interval
  df = math.exp(-r*dt) # discount factor per time interval
  # this will be used for discounting back cfs backwards from end point
 
  
  # binomial paramters
  u = math.exp(sigma*math.sqrt(dt)) # up-movement
  d = 1/u # down-movement
  q = (math.exp(r*dt)-d)/(u-d) # martiangle (risk-neutral probability)
  
  
  # Array Initialization for Index Levels
  mu = np.arange(M + 1)
  mu = np.resize(mu, (M + 1, M + 1))
  md = np.transpose(mu)
  mu = u ** (mu - md)
  md = d ** md
  S = S0 * mu * md

  # Valuation by Risk-Neutral Discounting
  
  if call:
    pv = np.maximum(S-K, 0) # present value array initialized with inner values
  else: 
    pv = np.maximum (K-S, 0) # present value array initialized with inner values
  z = 0
  for i in np.arange(M - 1, -1, -1): # backwards induction
    pv[0:M - z, i] = (q * pv[0:M - z, i + 1] + (1 - q) * pv[1:M - z + 1, i + 1]) * df
    z += 1
  Vo = pv[0,0]
  return Vo

"""
NOTE: please come back to it again

"""

In [0]:
print(" Value of Europen Call option is ${0:.2f}".format(CRR_value_option(105,100,1,0.05,.25,call=True)))
print(" Value of Europen Put option is ${0:.2f}".format(CRR_value_option(105,100,1,0.05,.25,call=False)))

 Value of Europen Call option is $15.65
 Value of Europen Put option is $5.78


In [0]:
# leaving out FFT for now

In [0]:
# using monte carlo to option call/put option value


def monte_carlo_option(So,K,T,r,sigma,paths=1000000,call=True):
  
  """
  simple monte carlo implementation to value call/put options
  
  Additionally, new paramater:
  paths : no. of paths to be considered
  
  Output:
  Vo: value of option i.e. call if call=True
  or, put call=False
  
  """
  import numpy as np
  z = np.random.standard_normal(paths) # this is the Z_t i.e. uncertanity used part of the last term
  df = np.exp(-r*T) # continously compunded discount factor
  
  # final simulated stock price based discrete form of stock price 
  S = np.array([So*np.exp((r-.5*sigma**2)*T+(sigma*np.sqrt(T)*z_t)) for z_t in z] )
  if call:
    payoff = np.maximum(S-K,0)
  else:
    payoff = np.maximum(K-S,0)
  Vo = df*np.sum(payoff)/paths # we are discounting back the average payoff that will be received at maturity
  # why not other way around? See Cauchy inequality
  
  return Vo

In [0]:
print('OPTION VALUATION USING MONTE CARLO TECHNIQUES')
print(" Value of Europen Call option is ${0:.2f}".format(monte_carlo_option(105,100,1,0.05,.25,paths=10000,call=True)))
print(" Value of Europen Put option is ${0:.2f}".format(monte_carlo_option(105,100,1,0.05,.25,paths=10000,call=False)))

OPTION VALUATION USING MONTE CARLO TECHNIQUES
 Value of Europen Call option is $15.74
 Value of Europen Put option is $5.64


In [0]:
# Interesting observation quite close
# Yet not there, but a great start indeed on this learning curve